<a href="https://colab.research.google.com/github/Awaistolu/Machine-learning-projects/blob/main/topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import string
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_csv('/content/npr.csv')

In [3]:
df.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [4]:
df.shape

(11992, 1)

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
ps = PorterStemmer()
lema = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
import re
def remove_digit(text):
  patten = re.compile('\d+')
  return patten.sub('',text)

In [7]:
# def transform_text(text):
#   text = text.lower()
#   text = nltk.word_tokenize(text)

#   y = []
  
#   for i in text:
#     if i.isalnum():
#       y.append(i)

#   text = y[:]
#   y.clear()
#   for i in text:
#     if i not in set(stopwords.words('english')) and i not in set(string.punctuation):
#       y.append(i)

#   # text = y[:]
#   # y.clear()
#   # for i in text:
#   #   y.append(ps.stem(i))

#   return ' '.join(y)


In [8]:
df['Article'] = df['Article'].apply(remove_digit)

In [10]:
# df['transformed_article'] = df['Article'].apply(transform_text)

In [9]:
df.head()

,Article
0,"In the Washington of , even when the policy ca..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at : p. m. ET, Russian President Vladi..."
4,"From photography, illustration and video, to d..."


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
cv = CountVectorizer(max_df=0.95,min_df=2,analyzer= 'word',stop_words='english')
dtm = cv.fit_transform(df['Article'])
dtm

<11992x53572 sparse matrix of type '<class 'numpy.int64'>'
	with 2968545 stored elements in Compressed Sparse Row format>

In [15]:
lda = LatentDirichletAllocation(n_components=7,random_state=10,n_jobs=-1)

In [16]:
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, n_jobs=-1, random_state=10)

In [17]:
print(lda.components_.shape)

(7, 53572)


In [18]:
single_topic = lda.components_[0]

In [19]:
single_topic.argsort()[-15:]

array([19883,  7970, 52006, 46008, 52784, 38650, 48256, 26245, 13820,
       47980, 41972, 25559, 27467, 35086, 41793])

In [20]:
import random

In [21]:
for i in range(10):
  random_word_id = random.randint(0,34519)
  print(cv.get_feature_names()[random_word_id])

manzanar
academies


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


assessments
circumstances
dimmed
boundary
panhandle
intercut
esvelt
mirrored


In [22]:
for index,topic in enumerate(lda.components_):
  print('-'*100)
  print(f'THE TOPIC OF #{index}')
  print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
  print('-'*100)
  print('\n')

----------------------------------------------------------------------------------------------------
THE TOPIC OF #0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['going', 'children', 'way', 'students', 'women', 'really', 'time', 'know', 'don', 'think', 'school', 'just', 'like', 'people', 'says']
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
THE TOPIC OF #1
['npr', 'rights', 'city', 'told', 'president', 'reports', 'law', 'country', 'government', 'state', 'court', 'people', 'police', 'says', 'said']
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
THE TOPIC OF #2
['make', 'family', 'world', 'city', 'home', 'water', 'day', 'time', 'new', 'years', 'just', 'people', 'like', 'food', 'says']
----------------------------------------------------------------------------------------------------


---------------------------------------------------

In [23]:
topic_results = lda.transform(dtm)

In [24]:
df['topic'] = topic_results.argmax(axis=1)

In [25]:
df.head()

,Article,topic
0,"In the Washington of , even when the policy ca...",3
1,Donald Trump has used Twitter — his prefe...,3
2,Donald Trump is unabashedly praising Russian...,3
3,"Updated at : p. m. ET, Russian President Vladi...",3
4,"From photography, illustration and video, to d...",6


In [26]:
df['topic'].value_counts()

5    2284
1    2171
3    2007
0    1652
6    1587
2    1319
4     972
Name: topic, dtype: int64

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier

In [28]:
tfidf = TfidfVectorizer(max_features=3000,max_df=0.95,min_df=2,analyzer='word',stop_words='english')
x = tfidf.fit_transform(df['Article']).toarray()

In [29]:
y = df.loc[:,['topic']]

In [30]:
x.shape

(11992, 3000)

In [31]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [32]:
gnb = GaussianNB()
bnb = BernoulliNB()
mnb = MultinomialNB()
rf = RandomForestClassifier()

In [33]:
# GaussianNB
gnb.fit(x_train,y_train)
y_pred1 = gnb.predict(x_test)
print('-'*50)
print(accuracy_score(y_test,y_pred1))
print('\n')
print(confusion_matrix(y_test,y_pred1))
print('-'*50)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


--------------------------------------------------
0.7828261775739892


[[214  11  18   6  42  39  16]
 [  3 343  20  17   0   9  12]
 [  9  14 197   1  27  22   8]
 [  5  26   2 312   0   6  19]
 [ 14   1  21   0 170   4   9]
 [ 23   6  27   6   1 407   0]
 [ 10  16  17  11  18   5 235]]
--------------------------------------------------


In [34]:
# BernoulliNB
bnb.fit(x_train,y_train)
y_pred2 = bnb.predict(x_test)
print('-'*50)
print(accuracy_score(y_test,y_pred2))
print('\n')
print(confusion_matrix(y_test,y_pred2))
print('-'*50)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


--------------------------------------------------
0.8182576073363902


[[224   8  13   6  39  41  15]
 [  9 359  11  14   0   2   9]
 [ 10  20 208   0  15  18   7]
 [  1  32   3 316   0   9   9]
 [ 12   2  13   0 176  10   6]
 [ 17   5  17   7   0 424   0]
 [ 12  17   9   4   9   5 256]]
--------------------------------------------------


In [35]:
# MultinomialNB
mnb.fit(x_train,y_train)
y_pred3 = mnb.predict(x_test)
print('-'*50)
print(accuracy_score(y_test,y_pred3))
print('\n')
print(confusion_matrix(y_test,y_pred3))
print('-'*50)

--------------------------------------------------
0.862025844101709


[[270  13   7   5   8  33  10]
 [  1 378   3  15   0   2   5]
 [  8  20 203   3   9  28   7]
 [  0  16   1 348   0   2   3]
 [ 20   4  19   0 158  13   5]
 [  8   6   8   9   0 439   0]
 [  9  10   9   7   4   1 272]]
--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
# RandomForestClassifier
rf.fit(x_train,y_train)
y_pred4 = rf.predict(x_test)
print('-'*50)
print(accuracy_score(y_test,y_pred4))
print('\n')
print(confusion_matrix(y_test,y_pred4))
print('-'*50)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


--------------------------------------------------
0.7728220091704877


[[236  18  11  13  10  42  16]
 [  7 354   0  33   0   5   5]
 [ 24  39 136   8  10  42  19]
 [  0  18   1 343   0   4   4]
 [ 32  11  12   0 139  14  11]
 [ 13  13   6  19   0 416   3]
 [ 14  28   5  27   3   5 230]]
--------------------------------------------------
